In [1]:
 # Dependencies and Setup
import pandas as pd

In [2]:
# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

In [12]:
# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


In [4]:
#District Summary

student_count = len(school_data_complete['Student ID'])
schools = school_data_complete.drop_duplicates(subset='school_name', keep='first')
school_count = len(schools)
total_budget=schools['budget'].sum()
av_math = round(school_data_complete['math_score'].mean(),2)
av_reading = round(school_data_complete['reading_score'].mean(),2)
passing_math_count = len(school_data_complete.loc[school_data_complete['math_score'] >= 70,:])
passing_math_perc =  round(passing_math_count/student_count*100,2)

passing_reading_count = len(school_data_complete.loc[school_data_complete['reading_score'] >= 70,:])
passing_reading_perc =  round(passing_reading_count/student_count*100,2)

passing_overall = school_data_complete.loc[(school_data_complete['math_score'] >= 70) | (school_data_complete['reading_score'] >= 70), :]
passing_overall_perc = round((len(passing_overall))/student_count*100,2)

district_summary_df = pd.DataFrame({
    'Total Schools' : school_count,
    'Total Students' : student_count,
    'Total Budget' : f"${total_budget:,}",
    'Average Math Score' : av_math,
    'Average Reading Score' : av_reading,
    '% Passing Math' : passing_math_perc,
    '% Passing Reading' : passing_reading_perc,
    '% Overall Passing' : passing_overall_perc
},
index=[0])
district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,"$24,649,428",78.99,81.88,74.98,85.81,95.61


In [5]:
schools = schools[['school_name','size','budget','type']]
schools = schools.sort_values(by='school_name')
schools = schools.set_index('school_name')
schools

,size,budget,type
school_name,,,
Bailey High School,4976,3124928,District
Cabrera High School,1858,1081356,Charter
Figueroa High School,2949,1884411,District
Ford High School,2739,1763916,District
Griffin High School,1468,917500,Charter
Hernandez High School,4635,3022020,District
Holden High School,427,248087,Charter
Huang High School,2917,1910635,District
Johnson High School,4761,3094650,District


In [6]:
grouped_schools_df = school_data_complete.groupby(['school_name'])
school_av_math = grouped_schools_df['math_score'].mean()
school_av_reading = grouped_schools_df['reading_score'].mean()
school_av_reading = school_av_reading.to_frame()
school_av_math = school_av_math.to_frame()


In [7]:
schools = schools.merge(school_av_reading, on='school_name', how='outer')
schools = schools.merge(school_av_math, on='school_name', how='outer')
schools

,size,budget,type,reading_score,math_score
school_name,,,,,
Bailey High School,4976,3124928,District,81.033963,77.048432
Cabrera High School,1858,1081356,Charter,83.975780,83.061895
Figueroa High School,2949,1884411,District,81.158020,76.711767
Ford High School,2739,1763916,District,80.746258,77.102592
Griffin High School,1468,917500,Charter,83.816757,83.351499
Hernandez High School,4635,3022020,District,80.934412,77.289752
Holden High School,427,248087,Charter,83.814988,83.803279
Huang High School,2917,1910635,District,81.182722,76.629414
Johnson High School,4761,3094650,District,80.966394,77.072464


In [8]:
passing_math = school_data_complete.loc[school_data_complete['math_score'] >= 70,:]
passing_math = passing_math.groupby(['school_name']).count()
passing_math = passing_math.rename(columns={'size': 'Passing Math'})

passing_reading = school_data_complete.loc[school_data_complete['reading_score'] >= 70,:]
passing_reading = passing_reading.groupby(['school_name']).count()
passing_reading = passing_reading.rename(columns={'size':'Passing Reading'})

spassing_overall = school_data_complete.loc[(school_data_complete['math_score'] >= 70) | (school_data_complete['reading_score'] >= 70), :]
spassing_overall = spassing_overall.groupby(['school_name']).count()
spassing_overall = spassing_overall.rename(columns={'size':'Passing Overall'})

passing_df = passing_math.merge(passing_reading, on='school_name', how='outer')
passing_df = passing_df.merge(spassing_overall, on='school_name', how='outer')
passing_df = passing_df[['Passing Math','Passing Reading', 'Passing Overall']]
schools_df = schools.merge(passing_df, on="school_name", how="outer")
passing_df

,Passing Math,Passing Reading,Passing Overall
school_name,,,
Bailey High School,3318,4077,4676
Cabrera High School,1749,1803,1855
Figueroa High School,1946,2381,2758
Ford High School,1871,2172,2556
Griffin High School,1371,1426,1467
Hernandez High School,3094,3748,4361
Holden High School,395,411,425
Huang High School,1916,2372,2727
Johnson High School,3145,3867,4463


In [9]:
schools_df = schools_df.rename(columns={
    'size':'Total Students',
    'budget': 'Total School Budget',
    'reading_score': 'Average Reading Score',
    'math_score': 'Average Math Score',
    'type': 'School Type'
})
schools_df

,Total Students,Total School Budget,School Type,Average Reading Score,Average Math Score,Passing Math,Passing Reading,Passing Overall
school_name,,,,,,,,
Bailey High School,4976,3124928,District,81.033963,77.048432,3318,4077,4676
Cabrera High School,1858,1081356,Charter,83.975780,83.061895,1749,1803,1855
Figueroa High School,2949,1884411,District,81.158020,76.711767,1946,2381,2758
Ford High School,2739,1763916,District,80.746258,77.102592,1871,2172,2556
Griffin High School,1468,917500,Charter,83.816757,83.351499,1371,1426,1467
Hernandez High School,4635,3022020,District,80.934412,77.289752,3094,3748,4361
Holden High School,427,248087,Charter,83.814988,83.803279,395,411,425
Huang High School,2917,1910635,District,81.182722,76.629414,1916,2372,2727
Johnson High School,4761,3094650,District,80.966394,77.072464,3145,3867,4463


In [ ]:
schools_df['Per Student Budget'] = schools_df['Total School Budget']/schools_df['Total Students']
schools_df = schools_df[['School Type','Total Students','Total School Budget','Per Student Budget','Average Math Score','Average Reading Score']]

schools_df['% Passing Math'] = round(schools_df['Passing Math']/schools_df['Total Students'],2)
schools_df['% Passing Reading'] = round(schools_df['Passing Reading']/schools_df['Total Students'],2)
schools_df['% Passing Overall'] = round(schools_df['Passing Overall']/schools_df['Total Students'],2)
schools_df